# Générons une prédiction valide pour le challenge

Imports nécessaires au notebook

In [1]:
import time
import re
import pandas as pd
import numpy as np
import nltk
import sklearn
import pickle
import tensorflow
from tensorflow import keras
import tensorflow_datasets as tfds

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

/home/cisd-calluau/nlp/python/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cisd-calluau/nlp/python/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cisd-calluau/nlp/python/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cisd-calluau/nlp/python/lib64/python3.6/site-pack

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Commencez par définir le nom de votre équipe termes alpha numériques sans espace

In [4]:
TEAM_NAME = "Wasabi"
assert re.match("^\w+$", TEAM_NAME) is not None, "Nom d'équipe invalide"

In [5]:
# Chargement des données d'entrainement
train = pd.read_csv("train.csv.gz")
train.head()

,idp,title,description,price,category
0,63610,4pairs silicone Oreillettes de remplacement du...,4pairs silicone Oreillettes de remplacement du...,4.90,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
1,580661,Dvd Rafaela Legouvello,Une aventure humaine hors du commun...,14.89,DVD - BLU-RAY > DVD > DVD DOCUMENTAIRE
2,90191,"Q2671a (H.309Ac) Toner Laser Hp Bleu (Cyan), ...","Q2671A (H.309AC) TONER LASER HP Bleu (Cyan),...",53.40,INFORMATIQUE > IMPRESSION - SCANNER > TONER - ...
3,1297725,Azalaïs ou La vie courtoise,Maryse Rouy,19.05,LIBRAIRIE > LITTERATURE > ROMANS HISTORIQUES
4,119129,Hamecon Coup Vmc Crystal 9408 Bronze (50 N°8),Hameçon coup Crystal 9408 bronzé VMC. Forme ...,7.59,SPORT > PECHE > HAMEÇON


In [6]:
# Chargement des données à prédire
test = pd.read_csv("test.csv.gz")
test.head()

,idp,title,description,price
0,4873686,Endoscope Sans Fil Wifi 1200 P Caméra D'inspec...,"Descriptions&nbsp;Résolution: 640 * 480, 1280 ...",16.85
1,6320508,Aider l'enfant dyslexique,Bernard Jumel 3e édition,16.90
2,6351042,Modà ̈Le réduit de véhicule de construction Li...,Modà¨le réduit de véhicule de construction Lie...,24.97
3,3853418,Lampenwelt lampadaire Led à intensité variable...,Ce lampadaire séduit par son design esthétique...,167.90
4,6373582,"Caméra vidéo caméscope, gordvec 2,7 Pouces écr...","""Caractéristiques : 2,7 pouces (16 : 9) LCD, l...",98.99


Combien y a t'il de catégories produits distinctes dans le dataset?

In [7]:
len(train.category.value_counts())

600

Ca fait beaucoup! Jusqu'ici vous n'aviez que 2 classes à prédire.

## Baseline : 
On va commencer par la prédiction la plus simple possible, une prédiction constante.

On prédit que toutes les catégories sont égales à la 1ère catégorie du dataset de train

In [29]:
category = train.iloc[0].category
print(f"On va toujours prédire: '{category}'")

On va toujours prédire: 'TV - VIDEO - SON > CASQUE - MICROPHONE - DICTAPHONE > CASQUE - ECOUTEURS'


sklearn a même un modèle tout fait pour se genre de cas d'usage : DummyClassifier

In [30]:
model = DummyClassifier(strategy='constant', constant=category)
model.fit(train["title"], train["category"])
y_pred = model.predict(test["title"])

et voici notre prédiction dans le bon format : 2 colonnes, l'identifiant produit suivi de sa catégorie

In [31]:
prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})
prediction.head()

,idp,category
0,4873686,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
1,6320508,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
2,6351042,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
3,3853418,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...
4,6373582,TV - VIDEO - SON > CASQUE - MICROPHONE - DICTA...


## La fonction de publication de vos prédictions

In [32]:
# predictions est un dataframe qui contient au moins les colonnes "idp" et "category"
# Ca écrit simplement le fichier dans le répertoire d'évaluation
def publish_results(predictions):
    now = int(time.time())
    assert re.match("^\w+$", TEAM_NAME) is not None
    filename = f"/home/cisd-jacq/projet/evaluation/prediction-{TEAM_NAME}-{now}.csv.gz"
    predictions[["idp", "category"]].to_csv(filename, index=False, compression="gzip")
    return filename.split("/")[-1]

Une fois que vous avez spécifié un nom d'équipe vous pouvez soumettre votre 1ère prédiction en exécutant la cellule suivante.

In [99]:
publish_results(prediction)

'prediction-Wasabi-1578308823.csv.gz'

## La fonction d'évaluation de l'erreur

Vous pouvez utiliser cette fonction pour estimer sur un sous ensemble du dataset train quelle est la précision de votre modèle.

In [33]:
# Plus ce score est grand moins on est content
def error(real_category, predicted_category):
    # On a trouvé la bonne catégorie
    if real_category == predicted_category:
        return 0
    
    # On extrait les sous catégories
    real_categories = real_category.split(" > ")
    pred_categories = predicted_category.split(" > ")
    # On flag les catégories adultes
    adult_categories = ['ADULTE - EROTIQUE', 'VIN - ALCOOL - LIQUIDES']
    real_is_adult = real_categories[0] in adult_categories
    pred_is_adult = pred_categories[0] in adult_categories
    
    # Attention non symmétrie de l'erreur !
    if real_is_adult and not pred_is_adult:
        error = 10_000
        return error
    
    # On identifie à quel niveau on s'est trompé
    for real, pred, error in zip(real_categories, pred_categories, [100, 10, 1]):
        if real != pred:
            return error
    raise ValueError("Catégories différentes, mais aucune différence trouvée")

# Une prédiction moins dummy

Le dataset est bien plus volumineux que les datasets utilisés en TP jusqu'à aujourd'hui.

Si vous vous lancez tête baissée à générer une prédiction vous allez attendre longtemps d'obtenir vos résultats. Il y a 2 raisons à cela : 
- Le nombre de lignes de l'ensemble train est très important
- Le nombre de classes à prédire est aussi très grand. Certains classifier multi-classe génèrent des classifier binaires de type One vs Rest. Si vous avez 600 classes vous allez apprendre 600 classifiers différents

Simplifions donc ces 2 problèmes.

## 1. Travaillons sur 10% du dataset de train

Commencons par itérer rapidement sur le jeu de données. Une fois qu'on aurra un modèle qui nous convient on pourra travailler sur plus de volumétrie.

Générez train_subset, un sample de train faisant 10% de sa taille

In [8]:
train_subset_size = int(0.1*train.shape[0])
chosen_idx = np.random.choice(train_subset_size, replace=False, size=train_subset_size)
train_subset = train.iloc[chosen_idx]

## 2. Limitons les classes

600 classes c'est beaucoup trop. 

On a vu que les catégories contiennent une hiérarchie "catégorie 1 > catégorie 2 > catégorie 3", commencons par travailler uniquement sur les catégories 1.

Générez la colonne "category_1" dans train_subset à partir de la colonne "category" qui contient uniquement la catégorie 1 du produit

In [9]:
def first_category(full_category):
    return full_category.split(">")[0].strip()

if 'category_1' in train_subset.columns:
    train_subset = train_subset.drop(columns=["category_1"])
train_subset.insert(train_subset.shape[1], "category_1", list(map(first_category, train_subset["category"])))

In [10]:
categories_freq = {}
for el in train_subset.iterrows():
    cat1 = el[1]["category_1"]
    if cat1 not in categories_freq:
        categories_freq[cat1] = {"max_freq" : 0, "max_cat" : el[1]["category"]}
    
    if el[1]["category"] not in categories_freq[cat1]:
        categories_freq[cat1][el[1]["category"]] = 1
    else :
        categories_freq[cat1][el[1]["category"]] += 1
        if categories_freq[cat1]["max_freq"] < categories_freq[cat1][el[1]["category"]]:
            categories_freq[cat1]["max_freq"] += 1 
            categories_freq[cat1]["max_cat"] = el[1]["category"]


Combien de "category_1" distinctes vous avez?

In [37]:
len(np.unique(train_subset["category_1"]))

30

C'est bien plus raisonnable pour commencer.

## 3. Oh oh il y a pleins de features.

Il y a pleins de features différentes.

- 2 champs de texte title et description 
- 1 champ float : price.

On va se contenter de travailler uniquement avec "title" pour commencer.

Générez les features : X_train

In [12]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

def process_text(line):
    tokens = nltk.word_tokenize(line)
    new_tokens = []
    pattern = '\w'
    for token in tokens:
        if len(token) > 1 and token not in stop_words and re.match(pattern, token):
            new_tokens.append(token.lower())
    return new_tokens

[nltk_data] Downloading package stopwords to /home/cisd-
[nltk_data]     calluau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
if 'title_tokens' in train_subset.columns:
    train_subset = train_subset.drop(columns=["title_tokens"])
train_subset.insert(train_subset.shape[1], "title_tokens", list(map(process_text, train_subset["title"])))
train_subset

,idp,title,description,price,category,category_1,title_tokens
14396,656706,Les chiens Corgi Pembroke Welsh Corgi Dessin d...,8 x 8 pouces spirale lié brillant dur recouver...,59.99,JEUX - JOUETS > LOISIRS CREATIFS ET ACTIVITES ...,JEUX - JOUETS,"[les, chiens, corgi, pembroke, welsh, corgi, d..."
4854,211136,Housse de clic clac Marie Rouge 120 à 1cm et 1...,Housse de clic clac très facile d'installation...,10.89,DECO - LINGE - LUMINAIRE > COUSSIN ET HOUSSE >...,DECO - LINGE - LUMINAIRE,"[housse, clic, clac, marie, rouge, 120, 1cm, 1..."
26797,45832,Vibox Armageddon Rm760 239 Pc Gamer Ordinateur...,AMD RX 460 4 Go (1x HDMI 2.0b) Intel i7 8700...,2135.00,INFORMATIQUE > ORDINATEUR DE BUREAU > UNITE CE...,INFORMATIQUE,"[vibox, armageddon, rm760, 239, pc, gamer, ord..."
3129,466777,Kit de douilles d'entraînement 193 pcs et outi...,Kit de douilles d'entraînement 193 pcs et outi...,110.19,AUTO - MOTO > OUTILLAGE > DOUILLES - EMBOUTS,AUTO - MOTO,"[kit, douilles, d'entraînement, 193, pcs, outi..."
12383,464399,Pour Oneplus Oneplus 6T : Batterie Externe de ...,Batterie externe de secours grande capacité (4...,39.99,TELEPHONIE - GPS > ACCESSOIRE TELEPHONE > BATT...,TELEPHONIE - GPS,"[pour, oneplus, oneplus, 6t, batterie, externe..."
...,...,...,...,...,...,...,...
3666,1238798,Usb domestique Inhalation Mosquito trapping no...,USB domestique Inhalation Mosquito trapping no...,8.02,ELECTROMENAGER > ENTRETIEN DES SOLS - ENTRETIE...,ELECTROMENAGER,"[usb, domestique, inhalation, mosquito, trappi..."
25468,157709,Batterie pour Jvc Gr Sxm48eg,Batterie Camescope Ni Mh. capacité: 2100 mAh p...,41.30,PHOTO - OPTIQUE > ACCESSOIRES PHOTO - OPTIQUE ...,PHOTO - OPTIQUE,"[batterie, jvc, gr, sxm48eg]"
10241,739675,Lumière portative légère de vélo de queue de b...,compte tenu du frein portatif de mini moto.b...,0.01,SPORT > CYCLES > ACCESSOIRES VELO - DECORATION...,SPORT,"[lumière, portative, légère, vélo, queue, bâti..."
28344,390476,Umiwe Épilateur à sourcils électrique Usb char...,Description: 1. Taille: 16.5x3cm. 2. Poids: 13...,9.69,ELECTROMENAGER > BEAUTE - BIEN-ETRE > EPILATEU...,ELECTROMENAGER,"[umiwe, épilateur, sourcils, électrique, usb, ..."


In [14]:
def assign_id_to_words(data, field):
    words_to_id = {}
    nextid = 0
    for row in data[field]:
        for word in row:
            if word not in words_to_id:
                words_to_id[word] = nextid
                nextid += 1
    return words_to_id

In [ ]:
if 'title_tokens' in test:
    test = test.drop(columns=["title_tokens"])
test.insert(test.shape[1], "title_tokens", list(map(process_text, test["title"])))

In [ ]:
words_to_id = assign_id_to_words(pd.concat(train_subset), "title_tokens")

In [ ]:
categories_to_id = {}
nextid = 0
for cat in train_subset["category_1"]:
    if cat not in categories_to_id:
         categories_to_id[cat] = nextid
         nextid += 1

In [ ]:
id_to_category = {v: k for k, v in categories_to_id.items()}

In [ ]:
def bag_of_words_to_vector(bag_of_words, words_to_id):
    features = np.zeros(len(words_to_id))
    for word in bag_of_words:
        if word in words_to_id:
            features[words_to_id[word]] = 1
    return features

In [ ]:
X_train = []
y_train = []
for row in train_subset.iterrows():
    X_train.append(bag_of_words_to_vector(row[1]["title_tokens"], words_to_id))
    y_train.append(categories_to_id[row[1]["category_1"]])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_train

In [ ]:
y_train

## 4. Calculez un modèle avec ces simplifications

Cf TP1 et TP2 : Entrainez une régression Logistique

In [ ]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)

## 5. Calculez votre prédiction

Appliquez votre modèle sur les données test

In [ ]:
confusion_matrix = sklearn.metrics.confusion_matrix(y_train, model.predict(X_train))


In [ ]:
sklearn.model_selection.cross_validate(model, X_train, y_train, cv=5)

## 6. Soumettez votre prédiction.

*Pas si vite* : Vous ne prédisez que la catégorie 1. Le script d'évaluation attend une catégorie complète...

C'est simple, pour chaque catégorie 1, choisissez la catégorie de votre choix qui commence par cette "categorie 1".

Modifiez votre prédiction, y_pred, en conséquence.

Soumettez votre prédiction :

In [105]:
test

,idp,title,description,price
0,4873686,Endoscope Sans Fil Wifi 1200 P Caméra D'inspec...,"Descriptions&nbsp;Résolution: 640 * 480, 1280 ...",16.85
1,6320508,Aider l'enfant dyslexique,Bernard Jumel 3e édition,16.90
2,6351042,Modà ̈Le réduit de véhicule de construction Li...,Modà¨le réduit de véhicule de construction Lie...,24.97
3,3853418,Lampenwelt lampadaire Led à intensité variable...,Ce lampadaire séduit par son design esthétique...,167.90
4,6373582,"Caméra vidéo caméscope, gordvec 2,7 Pouces écr...","""Caractéristiques : 2,7 pouces (16 : 9) LCD, l...",98.99
...,...,...,...,...
999995,6036736,Grille Pomme De Douche (5) Presto 90164,GRILLE POMME DE DOUCHE (5) PRESTO 90164 Ce s...,7.15
999996,761492,Brosse pinceau pour correcteurs et anti cernes,Brosse professionnelle de maquillage eetagrave...,9.90
999997,5165063,La ruine et le geste architectural,La ruine et le geste architectural Pierre Hy...,20.00
999998,3292782,Cen Robinet G1 2 robinet de bassin de lavabo b...,"☻☻Facile à installer, nécessite seulement une ...",31.84


In [106]:
X_test = []

for row in test.iterrows():
    X_test.append(bag_of_words_to_vector(row[1]["title_tokens"], words_to_id))

X_test = np.array(X_test)

KeyError: 'dyslexique'

In [98]:
X_test

In [98]:
y_temp = model.predict(X_test)
y_pred = list(map(lambda cat_id: categories_freq[id_to_category[cat_id]]["max_cat"], y_temp))

prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})

['BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL A MAIN > PINCE ELECTRICIEN',
 'MUSIQUE > CD > CD VARIETE INTERNATIONALE',
 'BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL A MAIN > PINCE ELECTRICIEN',
 'INFORMATIQUE > CONNECTIQUE - ALIMENTATION > CABLE AUDIO VIDEO',
 'DECO - LINGE - LUMINAIRE > ARTICLES - DECORATION DE FETE > BALLON DECORATIF - POMPE POUR BALLON',
 'HYGIENE - BEAUTE - PARFUM > CAPILLAIRE > BANDEAU - SERRE-TETE - HEADBAND - HAIRBAND',
 'JEUX - JOUETS > VEHICULE POUR ENFANT > ACCESSOIRE - PIECE DETACHEE VEHICULE',
 'TV - VIDEO - SON > LECTEUR MUSIQUE > LECTEUR MP3',
 'BAGAGERIE > MAROQUINERIE > POCHETTE A MAIN',
 'INSTRUMENTS DE MUSIQUE > MATERIEL DE JEU > PARTITION',
 'INFORMATIQUE > CONNECTIQUE - ALIMENTATION > CABLE AUDIO VIDEO',
 'CHAUSSURES - ACCESSOIRES > BASKET - SPORTSWEAR > BASKET',
 'BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL A MAIN > PINCE ELECTRICIEN',
 'ANIMALERIE > JOUET > JOUET',
 'BRICOLAGE - OUTILLAGE - QUINCAILLERIE > OUTIL A MAIN > PINCE ELECTRICIEN',

Vous pouvez regardez vos scores en exécutant le notebook [Leaderboard.ipynb](Leaderboard.ipynb). Les données sont mises à jour toutes les 30min.

# Sauvegardez votre modèle

Pour ne pas devoir recommencer à chaque fois tout ce dur labeur, et cette longue attente, vous pouvez sauvegarder votre modèle et votre vectorizer.

La prochaine fois vous n'aurez qu'à les recharger pour faire directement vos prédictions (c'est ce qui est attendu pour la soutenance, sinon le timing sera trop serré) 

La documentation : https://scikit-learn.org/stable/modules/model_persistence.html

In [ ]:
pickle.dump(model)

# Maintenant à vous de jouer

Vous pouvez commencer par travailler sur plus de volumétrie que 5% du dataset. Mais maintenant le challenge commence.

Si vous ne savez pas par où commencer suivez le déroulement des 2 premiers TPs, en prenant garde à la volumétrie. Ils sont disponibles dans le répertoire ~cisd-jacq/TP/

Contrairement aux TPs vous n'avez pas d'information sur les données de test. (Mis à part le score calculé toute les 30min).

Pour évaluer votre modèle et l'améliorer vous pouvez utiliser les données de train pour crééer un ensemble d'entrainement et un ensemble de validation. 

Vous pourrez alors évaluer plus rapidement vos modèles et identifier quelles sont les catégories sur lesquelles vous devez vous améliorer.

Il n'y a pas que la Régression Logistique dans la vie, essayez d'autres modèles. Je vous ai fait travailler avec sklearn, mais il existe aussi d'autres librairies.

Pour paralléliser vos calculs :
- multiprocessing : https://docs.python.org/3/library/multiprocessing.html
- dask : https://dask.org/ + https://distributed.dask.org/en/latest/ 

In [ ]:
encoder = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_generator, target_vocab_size=2**15)
encoder.save_to_file(vocab_filename)